In [1]:
test_input1 = """broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a"""

test_input2 = """broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output"""

In [2]:
input = open("inputs/20").read()

In [3]:
from collections import deque

d = {}

# for l in test_input1.splitlines():
# for l in test_input2.splitlines():
for l in input.splitlines():
    label, outputs = l.split(' -> ')

    outputs = outputs.split(', ')

    if label != 'broadcaster':
        module_type = label[0]
        label = label[1:]
    else:
        module_type = 'broadcaster'
    
    if module_type == '%':
        state = False
    elif module_type == '&':
        state = dict()
    else:
        state = None
    
    d[label] = [module_type, outputs, state]

# d['output'] = ['output', [], None]

# now let's go back over so we can make conjunctions aware of their inputs

for label, (module_type, outputs, state) in d.items():
    for o in outputs:
        if o in d:
            do = d[o]
            if do[0] == '&':
                do[-1][label] = 0

In [4]:
import networkx as nx

g = nx.DiGraph()

for label, (module_type, outputs, state) in d.items():
    g.add_node(label, module_type=module_type)
    for i, o in enumerate(outputs, start=1):
        g.add_edge(label, o, index=i)

In [5]:
g.number_of_edges(), g.number_of_nodes()

(109, 59)

In [6]:
from networkx.drawing.nx_agraph import to_agraph 

A = to_agraph(g)
A

<AGraph <Swig Object of type 'Agraph_t *' at 0x7fef8b67a190>>

In [7]:
for node in A.nodes():
    # node.attr['label'] = 'Custom Label'  # Set a custom label

    module_type = node.attr['module_type']
    node.attr['color'] = 'red' if module_type == '%' else 'blue' if module_type == '&' else 'black'

In [8]:
for edge in A.edges():
    edge.attr['label'] = edge.attr['index']

In [9]:
A.layout()
A.draw('my_graph.png', prog='dot')  # Save to file

In [10]:
def state_is_reset():
    for label, (module_type, outputs, state) in d.items():
        if module_type == '%':
            if state:
                return False

        elif module_type == '&':
            if any(state.values()):
                return False

    return True

In [11]:
from collections import Counter

# labels_to_monitor = set(['tx', 'dd', 'nz', 'ph'])
labels_to_monitor = set(['zq', 'qm', 'dc', 'jh'])

def push_button():
    # push button, sending a low pulse to the broadcaster

    signal_queue = deque()
    signal_queue.append(('button', 'broadcaster', 0))

    counts = []
    encounters = []

    num_iters = 0
    while len(signal_queue) > 0:
        num_iters += 1
        sender, label, signal = signal_queue.popleft()
        counts.append(signal)

        # if label == 'ls' and sender in labels_to_monitor and signal == 1:
        #     encounters.append((sender, label, signal, num_iters))

        if label not in d:
            continue

        module, outputs, state = d[label]

        if module == 'broadcaster':
            for o in outputs:
                signal_queue.append((label, o, signal))
        elif module == '%':
            if signal == 1:
                pass
            else:
                d[label][-1] = not d[label][-1]

                for o in outputs:
                    signal_queue.append((label, o, int(not state)))
        elif module == '&':
            state[sender] = signal

            out_signal = int(not all(state.values()))

            if not out_signal and label in labels_to_monitor:
                encounters.append((sender, label, state, signal, num_iters))

            for o in outputs:
                signal_queue.append((label, o, out_signal))

    # print(num_iters)
    return Counter(counts), state_is_reset(), encounters, num_iters

In [12]:
# rx is low when all of ls's inputs are high

In [13]:
import math
math.lcm(3779, 3889, 3907, 4051)

232605773145467

In [16]:
import itertools

# labels_to_monitor = ['tx', 'dd', 'nz', 'ph']
# for i in itertools.count(1000):
# for i in range(1000):
for i in range(1, 20_000):
    if i % 100_000 == 0:
        print(i)

    counts, is_reset, encounters, total_iters = push_button()

    if encounters:
        print(i, encounters, total_iters)

3779 [('ss', 'zq', {'ss': 0, 'dj': 0, 'fv': 0, 'vj': 0, 'rt': 0, 'hh': 0, 'nd': 0}, 1, 10)] 133
3889 [('kb', 'qm', {'lm': 0, 'bf': 0, 'pv': 0, 'sj': 0, 'ms': 0, 'lg': 0, 'kb': 0}, 1, 6)] 144
3907 [('qg', 'jh', {'qg': 0, 'xm': 0, 'dg': 0, 'nk': 0, 'xr': 0, 'kr': 0, 'zm': 0}, 1, 12)] 171
4051 [('vq', 'dc', {'db': 0, 'kl': 0, 'vq': 0, 'qv': 0, 'jn': 0, 'kf': 0, 'hq': 0, 'mr': 0, 'bh': 0}, 1, 9)] 162
7558 [('ss', 'zq', {'ss': 0, 'dj': 0, 'fv': 0, 'vj': 0, 'rt': 0, 'hh': 0, 'nd': 0}, 1, 10)] 133
7778 [('kb', 'qm', {'lm': 0, 'bf': 0, 'pv': 0, 'sj': 0, 'ms': 0, 'lg': 0, 'kb': 0}, 1, 6)] 145
7814 [('qg', 'jh', {'qg': 0, 'xm': 0, 'dg': 0, 'nk': 0, 'xr': 0, 'kr': 0, 'zm': 0}, 1, 12)] 173
8102 [('vq', 'dc', {'db': 0, 'kl': 0, 'vq': 0, 'qv': 0, 'jn': 0, 'kf': 0, 'hq': 0, 'mr': 0, 'bh': 0}, 1, 9)] 165
11337 [('ss', 'zq', {'ss': 0, 'dj': 0, 'fv': 0, 'vj': 0, 'rt': 0, 'hh': 0, 'nd': 0}, 1, 10)] 133
11667 [('kb', 'qm', {'lm': 0, 'bf': 0, 'pv': 0, 'sj': 0, 'ms': 0, 'lg': 0, 'kb': 0}, 1, 6)] 144
11721 [

In [14]:
# zq: 3779, 7558, 11337
# qm: 3889, 7778, 11667
# jh: 3907, 7814, 11721 
# dc: 4051, 8102, 12153

In [15]:
math.prod([3779, 3889, 3907, 4051])

232605773145467